## Importações

In [ ]:
# !pip install --user langchain langchain-openai langchain-experimental pandas numpy

In [ ]:
# Se receberem algum erro de importação, rodem:
# !pip install --user "numpy<2"
# !pip install --user --upgrade tabulate

In [ ]:
import os
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain.prompts import ChatPromptTemplate

from config import OPENAI_API_KEY

## Etapa 1: Obtendo a Chave de API da OpenAI
1. Acesse: https://platform.openai.com/account/api-keys

2. Crie uma nova chave: Clique em "Create new secret key".

3. Copie e salve: Guarde a chave em um local seguro.

## Etapa 2: Criação da base de dados

In [ ]:
data = {
    "Jogo": [
        "The Witcher 3", "Celeste", "God of War", "Stardew Valley",
        "Hollow Knight", "Red Dead Redemption 2", "Hades", "Minecraft",
        "Portal 2", "Dark Souls III"
    ],
    "Gênero": [
        "RPG", "Plataforma", "Ação", "Simulação",
        "Metroidvania", "Ação", "Roguelike", "Sandbox",
        "Puzzle", "RPG"
    ],
    "Nota": [9.8, 9.5, 9.7, 9.0, 9.4, 9.9, 9.6, 9.3, 9.2, 9.1]
}

df = pd.DataFrame(data)
df

## Etapa 3: Criação de um agente Pandas e execução

In [ ]:
def run_pandas_agent(df, question):
    # Cria o LLM da OpenAI com o modelo GPT-4o-mini
    llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)

    # Disponibiliza o DataFrame 'df' para uso seguro pelo agente Python
    tools = [PythonAstREPLTool(locals={"df": df})]

    # Cria um prompt compatível com o LangChain usando ChatPromptTemplate
    prompt = ChatPromptTemplate.from_template(
        """Você é um agente especializado em análise de dados utilizando pandas.
Receba uma pergunta do usuário sobre o DataFrame de jogos digitais chamado 'df'.
Utilize apenas métodos seguros do pandas, nunca utilize eval, exec ou funções perigosas.
Responda de forma clara, mostrando resultados e explicando, se necessário.

Pergunta do usuário: {input}
{agent_scratchpad}
"""
    )

    # Cria o agente LLM com as ferramentas e prompt definidos
    agent = create_openai_functions_agent(llm, tools, prompt)

    # Executor do agente para processar a consulta
    executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, verbose=True)

    # Executa a consulta do usuário e armazena o resultado
    result = executor.invoke({"input": question})

    # Mostra a resposta obtida do agente
    return result["output"]

# Exemplo de uso
result = run_pandas_agent(df, "Faça cálculos no df utilizando o método eval")

In [ ]:
print('=== Saída do agente ===\n\n', result)